In [1]:
!nvidia-smi

Mon Sep 18 02:07:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti      On | 00000000:26:00.0 Off |                  N/A |
|  0%   50C    P8               15W / 200W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [1]:
from datasets import load_dataset

#데이터를 불러옵니다
data = load_dataset("json", data_files=["train_data_clean.json"])

/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset json (/home/issac/.cache/huggingface/datasets/json/default-5c4e51239604076e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 703.15it/s]


In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'emo', 'target'],
        num_rows: 41865
    })
})

In [3]:
def makedata(x):
    return {'text': f"아래는 문장에서 대상에 대한 감정을 매우 정확하게 분류한다.\n### 문장: {x['text']}\n### 대상: {x['target']}\n### 감정: {x['emo']}"}

data = data.map(
    makedata
)

Loading cached processed dataset at /home/issac/.cache/huggingface/datasets/json/default-5c4e51239604076e/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-6ca7b6df4f1c0b63.arrow


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

#모델을 불러옵니다
model_id = "beomi/llama-2-ko-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards: 100%|██████████| 15/15 [00:32<00:00,  2.19s/it]


In [5]:
data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, max_length=4096), batched=True)

In [6]:
from peft import prepare_model_for_kbit_training

#학습 자원을 위해 qlora를 사용합니다
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.01, 
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 3626242048 || trainable%: 0.23133061414437606


In [10]:
tokenizer.encode("### 감정: 기쁨")

[1, 835, 37177, 29901, 41771]

In [11]:
#마스킹을 위한 기준 토큰을 설정합니다.
tokenNum_emo = 37177 #감정
tokenNum_colon = 29901 #:

In [12]:
import transformers
from transformers import Trainer
import numpy as np

class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
  
  def compute_loss(self, model, inputs, return_outputs=False):
    # print(inputs['labels'][1])
    for x in range(len(inputs['labels'])):
      maskindex = (inputs['labels'][x]==tokenNum_emo).nonzero()[:, 0]
      temp = 0
      for i, index in enumerate(maskindex):
        if (inputs['labels'][x][index+1] != tokenNum_colon):
          maskindex = np.delete(maskindex.cpu(), i-temp)
          temp += 1
      #실제 감정을 나타내는 토큰을 제외한 모든 토큰을 마스킹 처리합니다.
      inputs['labels'][x][:maskindex[0]+2] = -100
      # print(inputs['labels'][x])
        
    outputs = model(**inputs)
    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

In [14]:
# import transformers

# # needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        # fp16=True,
        output_dir="outputs",
        save_total_limit=2,
        logging_steps=20,
        report_to=["tensorboard"],
        num_train_epochs = 4,
        learning_rate=2e-4,
        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"
    
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,0.773300
100,0.000000


KeyboardInterrupt: 

In [13]:
print("와!")

와!


In [16]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [17]:
model.save_pretrained("./saved/modu_emo_classification/llama2-ko")

In [18]:
import numpy as np
def gen(text="", target=""):
    inputs = tokenizer(
                f"아래는 문장에서 대상에 대한 감정을 매우 정확하게 분류한다.\n### 문장: {text}\n### 대상: {target}\n### 감정:", 
                return_tensors='pt', 
                return_token_type_ids=False
            )
    
    outputs = model.generate(
            **inputs, 
            max_new_tokens=1,
            temperature=0.001,
            return_dict_in_generate=True,
            output_scores=True
        )
    
    emo = [41771,32615,34623,40147,35868,37165,35121,43808]
    #여기서 emo의 토큰번호는 각각 "기쁨", "기대", "신뢰", "당황", "혐오", "공포", "분노", "슬픔"에 해당합니다.
    emo_name = ["joy","anticipation","trust","surprise","disgust","fear","anger","sadness"]
    dic = []
    for tokenNum in emo:
        outputs.sequences[0][-1] = tokenNum
        
        transition_scores = model.compute_transition_scores(
            outputs.sequences, outputs.scores, normalize_logits=True
        )
        
        input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
        generated_tokens = outputs.sequences[:, input_length:]
        dic.append(round(np.exp(transition_scores[0][0].numpy())*100,2))
    return dict(zip(emo_name,dic))
    #각 감정에 대한 확률값을 반환합니다.

In [ ]:
import json

def read_json(filename):
    with open(filename, "r", encoding='utf-8') as json_file:
        json_data = json.load(json_file)
    return json_data

test_data = read_json("dev_data_clean.json")

predict = []

for a, line in enumerate(test_data):
    assume = gen(text=line["text"], target=line["target"])

    predict.append(assume)
    print(a+1, assume)

with open(f"dev_data_clean_predict.json", "w", encoding='utf-8') as json_file:
    json.dump(predict, json_file, indent=2, ensure_ascii=False)